In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1)  # input -? Output? RF
        self.bn1 = nn.BatchNorm2d(10)  # BatchNorm after conv1
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(20)  # BatchNorm after conv2
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 10, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(10)  # BatchNorm after conv3
        self.conv4 = nn.Conv2d(10, 10, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(10)  # BatchNorm after conv4
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3)
        self.bn5 = nn.BatchNorm2d(10)  # BatchNorm after conv5
        self.conv6 = nn.Conv2d(10, 10, 3)
        self.bn6 = nn.BatchNorm2d(10)  # BatchNorm after conv6
        self.conv7 = nn.Conv2d(10, 120, 3)
        self.bn7 = nn.BatchNorm2d(120)  # BatchNorm after conv7
        self.fc = nn.Linear(120, 10)

        # Dropout layers
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.dropout(x)
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = self.dropout(x)
        x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))
        x = self.dropout(x)
        x = F.relu(self.bn7(self.conv7(x)))
        x = x.view(x.size(0), -1)  # flatten tensor
        x = self.fc(x)
        x = self.dropout(x)

        return F.log_softmax(x, dim=1)


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
            Conv2d-3           [-1, 20, 28, 28]           1,820
       BatchNorm2d-4           [-1, 20, 28, 28]              40
         MaxPool2d-5           [-1, 20, 14, 14]               0
           Dropout-6           [-1, 20, 14, 14]               0
            Conv2d-7           [-1, 10, 14, 14]           1,810
       BatchNorm2d-8           [-1, 10, 14, 14]              20
            Conv2d-9           [-1, 10, 14, 14]             910
      BatchNorm2d-10           [-1, 10, 14, 14]              20
        MaxPool2d-11             [-1, 10, 7, 7]               0
          Dropout-12             [-1, 10, 7, 7]               0
           Conv2d-13             [-1, 10, 5, 5]             910
      BatchNorm2d-14             [-1, 1

In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.0MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 434kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.01MB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.52MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.025, momentum=0.9)
scheduler= StepLR(optimizer, step_size=9, gamma=0.15)


for epoch in range(1, 20):
    print(f"Epoch {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1, Learning Rate: 0.025


loss=0.12566810846328735 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 18.91it/s]



Test set: Average loss: 0.0634, Accuracy: 9786/10000 (98%)

Epoch 2, Learning Rate: 0.025


loss=0.12301745265722275 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.13it/s]



Test set: Average loss: 0.0417, Accuracy: 9871/10000 (99%)

Epoch 3, Learning Rate: 0.025


...


Test set: Average loss: 0.0197, Accuracy: 9945/10000 (99%)

Epoch 18, Learning Rate: 0.00375


loss=0.12088511139154434 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 19.78it/s]



Test set: Average loss: 0.0196, Accuracy: 9943/10000 (99%)

Epoch 19, Learning Rate: 0.0005625


loss=0.06774037331342697 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.35it/s]



Test set: Average loss: 0.0204, Accuracy: 9940/10000 (99%)

